In [223]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr
import os
import yaml
import itertools
import shutil
from datetime import datetime
import sys

In [224]:
reference = 'SvenssonLinks'

sv_links = pd.read_excel('/Users/quinnmackay/Documents/GitHub/BICC/Data Storage/Tiepoints/Svensson Links NGRIP-NEEM-GIP-GISP-EDML-EDC-WD-DF-TALDICE.xls', sheet_name=1, skiprows=0, usecols=[3,8], names=['GISP2', 'TALDICE'])
sv_links = sv_links[:-5]

sv_links.dropna(inplace=True)

comment_add = 'Svensson Match Points, unpublished.'

folder='Bipolar'

sv_links

,GISP2,TALDICE
0,1575.712,606.61970
1,1597.200,619.24265
2,1640.880,645.21380
3,1653.880,654.69000
4,1663.790,662.19850
...,...,...
179,2480.283,1242.23860
181,2482.037,1243.21000
182,2485.052,1245.26000
185,2493.250,1249.75770


In [225]:
#load cores list from params
params = f'/Users/quinnmackay/Documents/GitHub/BICC/Antarctic Chronology Accuracy Project/{folder}/parameters.yml'
with open(params, 'r') as f:
    first_line = f.readline()
params_load = yaml.safe_load(first_line)
list_sites = params_load['list_sites']

# get all link combos
pairs = [f"{a}-{b}" for a, b in itertools.combinations(list_sites, 2)]

#get all combos possible from the svensson links
subset = list(sv_links.columns)
valid_pairs = [p for p in pairs if all(site in subset for site in p.split('-'))]

In [226]:
#now, create all the link files
sv_synchros = {}
sv_comments = {}
for pair in valid_pairs:
    site_a, site_b = pair.split('-')

    # Drop NaN rows to get only valid shared tiepoints
    sv_synchros[pair] = sv_links[[site_a, site_b]].dropna()
    sv_synchros[pair].columns = ['depth1', 'depth2']
    sv_synchros[pair]['reference'] = reference  # Load reference string
    sv_comments[pair] = [comment_add]

#now load existing synchros
os.chdir(f'/Users/quinnmackay/Documents/GitHub/BICC/Antarctic Chronology Accuracy Project/{folder}')
existing_synchros = {}

for pair in valid_pairs:

    folder_path = pair
    file_path = os.path.join(folder_path, f"{reference}_{pair}.txt")
    os.makedirs(folder_path, exist_ok=True)

    if os.path.exists(file_path):
        raise FileExistsError(f"File already exists: {file_path}")

    combined = sv_synchros[pair].copy(deep=True) # svensson links load
    new_comments = sv_comments.get(pair, [])

    with open(file_path, 'w') as f:
        # Write new comments first
        for line in new_comments:
            f.write(f"#{line}\n")
        # Write merged DataFrame below
        combined.to_csv(f, sep='\t', index=False)

    print(f"Saving new synchro to {file_path}")

Saving new synchro to GISP2-TALDICE/SvenssonLinks_GISP2-TALDICE.txt
